# 参数初始化
参数初始化对模型具有较大的影响，不同的初始化方式可能会导致截然不同的结果，所幸的是很多深度学习的先驱们已经帮我们探索了各种各样的初始化方式，所以我们只需要学会如何对模型的参数进行初始化的赋值即可。

PyTorch 的初始化方式并没有那么显然，如果你使用最原始的方式创建模型，那么需要定义模型中的所有参数，当然这样可以非常方便地定义每个变量的初始化方式。但是对于复杂的模型，这并不容易，而且推荐使用 Sequential 和 Module 来定义模型，所以这个时候就需要知道如何来自定义初始化方式。

## 1. 使用 NumPy 来初始化
因为 PyTorch 是一个非常灵活的框架，理论上能够对所有的 Tensor 进行操作，所以我们能够通过定义新的 Tensor 来初始化，直接看下面的例子

In [1]:
import numpy as np
import torch
from torch import nn

In [2]:
# 定义一个 Sequential 模型
net1 = nn.Sequential(
    nn.Linear(30, 40),
    nn.ReLU(),
    nn.Linear(40, 50),
    nn.ReLU(),
    nn.Linear(50, 10)
)

In [3]:
# 访问第一层的参数
w1 = net1[0].weight
b1 = net1[0].bias

In [4]:
print(w1)

Parameter containing:
tensor([[-0.0784,  0.1559,  0.0451,  ...,  0.0432,  0.0325, -0.0626],
        [ 0.0436,  0.0976,  0.1529,  ..., -0.1601, -0.1227, -0.0831],
        [ 0.0890,  0.0343,  0.1744,  ..., -0.0332,  0.0897,  0.0002],
        ...,
        [-0.1447, -0.0411, -0.0851,  ...,  0.0117,  0.1457,  0.0585],
        [ 0.1642,  0.0744, -0.1118,  ...,  0.0623, -0.0591,  0.0512],
        [-0.1610,  0.0070,  0.0184,  ..., -0.1529, -0.0314,  0.1748]],
       requires_grad=True)


注意，这是一个 Parameter，也就是一个特殊的 Variable，我们可以访问其 `.data`属性得到其中的数据，然后直接定义一个新的 Tensor 对其进行替换，我们可以使用 PyTorch 中的一些随机数据生成的方式，比如 `torch.randn`，如果要使用更多 PyTorch 中没有的随机化方式，可以使用 numpy

In [4]:
# 定义一个 Tensor 直接对其进行替换
net1[0].weight.data = torch.from_numpy(np.random.uniform(3, 5, size=(40, 30)))

In [5]:
print(net1[0].weight)

Parameter containing:
tensor([[3.3959, 3.0581, 4.1203,  ..., 3.3177, 3.4147, 4.5274],
        [4.1471, 3.9379, 3.9466,  ..., 4.1818, 4.2266, 4.0724],
        [4.9004, 4.0929, 4.7103,  ..., 3.6045, 4.6051, 3.2360],
        ...,
        [3.9905, 4.6050, 4.6511,  ..., 4.6798, 4.7991, 3.2708],
        [3.1855, 3.1292, 4.2139,  ..., 4.5036, 3.9371, 4.7396],
        [4.5988, 3.0121, 3.4215,  ..., 3.1637, 3.8030, 4.8393]],
       dtype=torch.float64, requires_grad=True)


可以看到这个参数的值已经被改变了，也就是说已经被定义成了我们需要的初始化方式，如果模型中某一层需要我们手动去修改，那么我们可以直接用这种方式去访问，但是更多的时候是模型中相同类型的层都需要初始化成相同的方式，这个时候一种更高效的方式是使用循环去访问，比如

In [6]:
for layer in net1:
    if isinstance(layer, nn.Linear): # 判断是否是线性层
        param_shape = layer.weight.shape
        layer.weight.data = torch.from_numpy(np.random.normal(0, 0.5, size=param_shape)) 
        # 定义为均值为 0，方差为 0.5 的正态分布

一种非常流行的初始化方式叫 Xavier，方法来源于 2010 年的一篇论文 [Understanding the difficulty of training deep feedforward neural networks](http://proceedings.mlr.press/v9/glorot10a.html)，其通过数学的推到，证明了这种初始化方式可以使得每一层的输出方差是尽可能相等。这种初始化的公式为：

$$
w\ \sim \ Uniform[- \frac{\sqrt{6}}{\sqrt{n_j + n_{j+1}}}, \frac{\sqrt{6}}{\sqrt{n_j + n_{j+1}}}]
$$

其中 $n_j$ 和 $n_{j+1}$ 表示该层的输入和输出数目，所以请尝试实现以下这种初始化方式

对于 Module 的参数初始化，其实也非常简单，如果想对其中的某层进行初始化，可以直接像 Sequential 一样对其 Tensor 进行重新定义，其唯一不同的地方在于，如果要用循环的方式访问，需要介绍两个属性，children 和 modules，下面我们举例来说明

In [7]:
class sim_net(nn.Module):
    def __init__(self):
        super(sim_net, self).__init__()
        self.l1 = nn.Sequential(
            nn.Linear(30, 40),
            nn.ReLU()
        )
        
        self.l1[0].weight.data = torch.randn(40, 30) # 直接对某一层初始化
        
        self.l2 = nn.Sequential(
            nn.Linear(40, 50),
            nn.ReLU()
        )
        
        self.l3 = nn.Sequential(
            nn.Linear(50, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.l1(x)
        x =self.l2(x)
        x = self.l3(x)
        return x

In [8]:
net2 = sim_net()

In [9]:
# 访问 children
for i in net2.children():
    print(i)

Sequential(
  (0): Linear(in_features=30, out_features=40, bias=True)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=40, out_features=50, bias=True)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=50, out_features=10, bias=True)
  (1): ReLU()
)


In [10]:
# 访问 modules
for i in net2.modules():
    print(i)

sim_net(
  (l1): Sequential(
    (0): Linear(in_features=30, out_features=40, bias=True)
    (1): ReLU()
  )
  (l2): Sequential(
    (0): Linear(in_features=40, out_features=50, bias=True)
    (1): ReLU()
  )
  (l3): Sequential(
    (0): Linear(in_features=50, out_features=10, bias=True)
    (1): ReLU()
  )
)
Sequential(
  (0): Linear(in_features=30, out_features=40, bias=True)
  (1): ReLU()
)
Linear(in_features=30, out_features=40, bias=True)
ReLU()
Sequential(
  (0): Linear(in_features=40, out_features=50, bias=True)
  (1): ReLU()
)
Linear(in_features=40, out_features=50, bias=True)
ReLU()
Sequential(
  (0): Linear(in_features=50, out_features=10, bias=True)
  (1): ReLU()
)
Linear(in_features=50, out_features=10, bias=True)
ReLU()


通过上面的例子，看到区别了吗?

children 只会访问到模型定义中的第一层，因为上面的模型中定义了三个 Sequential，所以只会访问到三个 Sequential，而 modules 会访问到最后的结构，比如上面的例子，modules 不仅访问到了 Sequential，也访问到了 Sequential 里面，这就对我们做初始化非常方便，比如

In [12]:
for layer in net2.modules():
    if isinstance(layer, nn.Linear):
        param_shape = layer.weight.shape
        layer.weight.data = torch.from_numpy(np.random.normal(0, 0.5, size=param_shape)) 

这上面实现了和 Sequential 相同的初始化，同样非常简便

## 2. `torch.nn.init`
因为 PyTorch 灵活的特性，可以直接对 Tensor 进行操作从而初始化，PyTorch 也提供了初始化的函数帮助我们快速初始化，就是 `torch.nn.init`，其操作层面仍然在 Tensor 上，下面我们举例说明

In [11]:
from torch.nn import init

In [12]:
print(net1[0].weight)

Parameter containing:
tensor([[-0.6533,  0.2052, -0.8449,  ..., -0.3461,  0.2085, -0.4455],
        [-0.5407,  0.6830, -0.7459,  ...,  0.2288,  0.1667, -0.0117],
        [ 0.0083,  0.6172,  0.0709,  ..., -0.8467, -0.1345,  0.3181],
        ...,
        [ 0.8485,  0.4565, -1.2186,  ..., -0.0108, -0.6460, -1.1651],
        [ 0.6727, -0.0261, -0.0799,  ..., -0.2073,  0.6962, -0.2177],
        [-0.1508, -1.1786, -0.4600,  ...,  0.2801, -0.1144,  0.4722]],
       dtype=torch.float64, requires_grad=True)


In [13]:
init.xavier_uniform_(net1[0].weight) # 这就是上面我们讲过的 Xavier 初始化方法，PyTorch 直接内置了其实现

Parameter containing:
tensor([[-0.1845,  0.0214, -0.2105,  ..., -0.0699,  0.0102, -0.1025],
        [-0.1395,  0.1330, -0.2138,  ..., -0.0085, -0.1500, -0.0176],
        [ 0.0657,  0.2689, -0.2434,  ...,  0.1945, -0.2152,  0.0809],
        ...,
        [-0.1298, -0.2575, -0.2274,  ...,  0.0977,  0.1394,  0.2270],
        [-0.0504, -0.1707, -0.1723,  ...,  0.1080,  0.1930, -0.2019],
        [ 0.1332,  0.2120,  0.2454,  ..., -0.1652,  0.1334,  0.2223]],
       dtype=torch.float64, requires_grad=True)

In [14]:
print(net1[0].weight)

Parameter containing:
tensor([[-0.1845,  0.0214, -0.2105,  ..., -0.0699,  0.0102, -0.1025],
        [-0.1395,  0.1330, -0.2138,  ..., -0.0085, -0.1500, -0.0176],
        [ 0.0657,  0.2689, -0.2434,  ...,  0.1945, -0.2152,  0.0809],
        ...,
        [-0.1298, -0.2575, -0.2274,  ...,  0.0977,  0.1394,  0.2270],
        [-0.0504, -0.1707, -0.1723,  ...,  0.1080,  0.1930, -0.2019],
        [ 0.1332,  0.2120,  0.2454,  ..., -0.1652,  0.1334,  0.2223]],
       dtype=torch.float64, requires_grad=True)


可以看到参数已经被修改了

`torch.nn.init` 提供了更多的内置初始化方式，避免了重复去实现一些相同的操作。

上面讲了两种初始化方式，其实它们的本质都是一样的，就是去修改某一层参数的实际值，而 `torch.nn.init` 提供了更多成熟的深度学习相关的初始化方式。
